# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [2]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [3]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [4]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [44]:
def foldl2(f, x0, lst):
    return foldr(lambda x, y: f(y, x), x0, lst[::-1])

In [45]:
def foldr2(f, x0, lst):
    return foldl(lambda x, y: f(y,x), x0, lst[::-1])

In [46]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно придумать как можно более оптимальный алгоритм и оценить его сложность. 

In [6]:
def check_inv(a, b):
    if len(a) < len(b):
        print("Неподходящие аргументы, первая строка должна быть не короче второй.")
        return 
    difference = 0
    count_dict = dict.fromkeys([letter for letter in b], 0)
    for letter in b:
        count_dict[letter] -= 1
        if count_dict[letter] == -1:
            difference += 1
    for _ in range(len(b)):
        if a[_] in count_dict.keys():
            count_dict[a[_]] += 1
            if count_dict[a[_]] == 1:
                difference += 1
            elif count_dict[a[_]] == 0:
                difference -= 1
    if difference == 0:
        print(f"Успешно! Индексы вхождения: ({0}, {len(b)})")
        return 0, len(b)
    for index, letter in enumerate(a[len(b):]):
        if letter in count_dict.keys():
            count_dict[letter] += 1
            if count_dict[letter] == 1:
                difference += 1
            elif count_dict[letter] == 0:
                difference -= 1
        if a[index] in count_dict.keys():
            count_dict[a[index]] -= 1
            if count_dict[a[index]] == -1:
                difference += 1
            elif count_dict[a[index]] == 0:
                difference -= 1
        if difference == 0:
            print(f"Успешно! Индексы вхождения: ({index + 1}, {index + len(b)})")
            return index+1, index + len(b)
    print("В первой строке нет перестановок второй.")
        

In [8]:
check_inv("trbaabc", "aab")

Успешно! Индексы вхождения: (2, 4)


(2, 4)

In [ ]:
# Сложность: O(len(a) + len(b))

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [135]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left is not None:
            yield from self.left
        if self.right is not None:
            yield from self.right
        if self.left is None and self.right is None:
            yield self.value
    
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return f"Leaf's value: {self.value}, left leaf's value: {self.left}, right leaf's value: {self.right}"
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [8]:
import re

def do_math(expr):
    if expr[2] == "*":
        return str(int(expr[1]) * int(expr[3]))
    elif expr[2] == "/":
        return str(int(expr[1]) // int(expr[3]))
    elif expr[2] == "+":
        return str(int(expr[1]) + int(expr[3]))
    elif expr[2] == "-":
        return str(int(expr[1]) - int(expr[3]))


def calc(expr):
    expr = expr.replace(" ", "")
    parentheses = True
    while parentheses:
        try:
            start_index = expr.index("(")
            end_index = expr.index(")")
            expr = expr[:start_index] + str(calc(expr[start_index+1:end_index])) + expr[end_index+1:]
        except ValueError:
            parentheses = False
    match = re.search(r"\d+[*/]\d+", expr)
    while match is not None:
        expr = re.sub(r"(\d+)([*/])(\d+)", do_math, expr)
        match = re.search(r"\d+[*/]\d+", expr)
    match = re.search(r"\d+[+\-]\d+", expr)
    while match is not None:
        expr = re.sub(r"(\d+)([+\-])(\d+)", do_math, expr)
        match = re.search(r"\d+[+\-]\d+", expr)
    return int(expr)
            
calc('2 * (15 - 3 * 4) - 2') == 4

True